<a href="https://colab.research.google.com/github/Yokitha-07/Zeroshot-prompting/blob/main/ZeroShot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import time
from google.colab import userdata
# Replace 'your_file.json' with the actual path to your JSON file
try:
    df_from_json = pd.read_json('/content/ticket_dataset.json')
except FileNotFoundError:
    print("Error: 'your_file.json' not found. Please upload the file or provide the correct path.")

In [20]:
from google import genai
from google.genai import types
from pydantic import BaseModel
from typing import List

# 1. Configure the client (using env var or explicitly)
api_key = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=api_key)

In [21]:
# 2. Define your schema using Pydantic
class ClassificationOutput(BaseModel):
    class_: str
    reason: str
    confidence: int

In [22]:
# Zero-shot Example Prompting
text_to_classify = "Checkout intermittently fails with 'try again later' when our traffic is routed to your EU replica region. Uptime looks fine—could this be regional latency?"

prompt_zero_shot = f"""
Classify the following text into one of these categories:
Billing: Any task relate to bills , ammount missmatch
Account Access:
Technical Issue:
Feature Request, Spam.
Provide the output as a JSON object with three keys: "class", "reason", and "confidence".

Output: {{"class": "<One of 5 type>", "reason": "<The text describes reason for selection>", "confidence": 0-100}}

Text: "{text_to_classify}"
"""

# 4. Call Gemini via the new SDK with schema enforcement
response_zero_shot = client.models.generate_content(
    model="gemini-2.5-flash",  # or whichever model you have access to
    contents=[prompt_zero_shot],
    config=types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=ClassificationOutput,
        temperature=0.5,
        max_output_tokens=5000,
    ),
)

print("\nZero-shot Example Prompting Output:")
print(response_zero_shot.text)


Zero-shot Example Prompting Output:
{"class_": "Technical Issue", "reason": "The text describes intermittent checkout failures, an error message 'try again later', and investigates potential regional latency issues related to traffic routing, all of which are technical problems.", "confidence": 95}
